In [ ]:
### import modules
import illustris_python as il
import multiprocessing as mp
import numpy as np
import h5py
import rohr_utils as ru
import os
import yaml
import glob
import argparse

class Configuration(dict):
    """
    Class to store all relevant information for a given simulation and sample set.
    """
    __slots__ = ()
    
    @classmethod
    def from_yaml(cls, fname):
        """ Load from a yaml file. """
        with open(fname, 'r') as fin:
            config = yaml.load(fin, yaml.SafeLoader)
        return cls(config)
    
    @classmethod
    def from_str(cls, text: str):
        """ Load from yaml-like string """
        config = yaml.load(text, yaml.SafeLoader)
        return cls(config)
    
    def __getattr__(self, key):
        """ Direct access as an attribute """
        try:
            return self[key]
        except KeyError:
            raise AttributeError(key)
            
    def __setattr__(self, key, value):
        """ Set additional attributes. """
        self[key] = value
        return


    def add_vals(self):
        """ Add additional attributes """
        
        #self = argparse_Config(self)
        
        self.basePath = ru.loadbasePath(self.sim)
        self.outdirec, self.outfname = return_outdirec_outfname(self)
        # for backwards compatibility
        self.GRPfname = self.outfname
        self.taufname = return_taufname(self)
        
        self.Header = il.groupcat.loadHeader(self.basePath, self.max_snap)
        self.h = self.Header['HubbleParam']
        
        SnapNums = np.arange(self.max_snap, self.min_snap-1, -1)
        Times = np.zeros(SnapNums.size, dtype=float)
        BoxSizes = Times.copy()
        for i, SnapNum in enumerate(SnapNums):
            header = il.groupcat.loadHeader(self.basePath, SnapNum)
            Times[i] = header['Time']
            BoxSizes[i] = header['BoxSize'] * Times[i] / self.h
        self.SnapNums = SnapNums
        self.Times = Times
        self.BoxSizes = BoxSizes
        zs, cosmictimes = ru.timesfromsnap(self.basePath, SnapNums)
        self.Redshifts = zs
        self.CosmicTimes = cosmictimes / 1.0e9 # Gyr
        
        self.gas_ptn = il.util.partTypeNum('gas')
        self.dm_ptn = il.util.partTypeNum('dm')
        self.tracer_ptn = il.util.partTypeNum('tracer')
        self.star_ptn = il.util.partTypeNum('star')
        self.bh_ptn = il.util.partTypeNum('bh')
        self.bary_ptns = [self.gas_ptn,
                          self.star_ptn,
                          self.bh_ptn]
                          
        self.Mstar_lolim = return_Mstar_lolim(self)
        
        # set the subfindIDs and accompanying snapnums of interest
        # first check if the tau_dict already exists
        full_taufname = self.outdirec + self.taufname
        if (self.zooniverse_flag):
            SnapNums_SubfindIDs, SubfindIDs = initialize_zooniverseindices(self)
        elif os.path.isfile(full_taufname):
            # yes, so just load the SubfindID (at z=0) for these
            print('File %s exists. Using SubfindIDs and SnapNums from there.'%(full_taufname))
            with h5py.File(full_taufname, 'r') as f:
                Group = f['Group']
                SubfindIDs = Group['SubfindID'][:]
                SnapNums_SubfindIDs = np.zeros(SubfindIDs.size, dtype=int) + 99
                f.close()
        # does the GRP file exist? If so, then use this file. 
        # Note that this assumes that the branches exist at z=0
        elif os.path.isfile(self.outdirec + self.GRPfname):
            print('File %s exists. Using SubfindIDs and SnapNums from there.'%(self.outdirec + self.GRPfname))
            with h5py.File(self.outdirec + self.GRPfname) as f:
                SubfindIDs = np.zeros(len(f.keys()), dtype=int) - 1
                SnapNums_SubfindIDs = SubfindIDs.copy() + 100
                for i, key in enumerate(f.keys()):
                    group = f[key]
                    SubfindIDs[i] = group['SubfindID'][0]
                f.close()
            if np.min(SubfindIDs) < 0:
                raise ValueError('Not all SubfindIDs are >=0 at z=0.')
        # no tau or GRP files, so initialize SnapNums and SubfindIDs here
        else:
            # based on the simulation and flags, find the appropriate initialziation function
            print('Files %s and %s do not exist. Initializing SubfindIDs and SnapNums elsewhere.'%(full_taufname,
                                                                                                   self.outdirec + self.GRPfname))
            # TNG-Cluster?
            if (self.TNGCluster_flag):
                SnapNums_SubfindIDs, SubfindIDs = initialize_TNGCluster_subfindindices(self)

            # all centrals with M200c > 10.**(11.15)?
            elif (self.centrals_flag):
                SnapNums_SubfindIDs, SubfindIDs = initialize_central_subfindindices(self)
                              
            # all subhalos?
            elif (self.allsubhalos_flag):
                SnapNums_SubfindIDs, SubfindIDs = initialize_allsubhalos(self)
              
            # general satellites?
            else:
                SnapNums_SubfindIDs, SubfindIDs = initialize_subfindindices(self)
                
                
        self.SubfindIDs = SubfindIDs
        self.SnapNums_SubfindIDs = SnapNums_SubfindIDs

        if self.tracers_flag:
            if self.zooniverse_flag:
                self.tracer_outdirec = '../Output/%s_tracers_zooniverse/'%(self.sim)
            else:
                self.tracer_outdirec = '../Output/%s_tracers/'%(self.sim)

            if not os.path.isdir(self.tracer_outdirec):
                print('Directory %s does not exist. Creating'%self.tracer_outdirec)
                os.system('mkdir %s'%self.tracer_outdirec)
            else:
                print('Directory %s exists. Potentially overwriting files.'%self.tracer_outdirec)
                
        self.zooniverse_keys = [self.ins_key, self.jel_key, self.non_key]
        self.subfindsnapshot_flags = [self.in_tree_key, self.central_key,
                                      self.in_z0_host_key, self.host_m200c_key]
        self.subfind_flags = [self.classified_flag, self.central_z0_flag,
                              self.backsplash_z0_flag, self.backsplash_prev_flag,
                              self.preprocessed_flag]
                              
        # tau dictionary keys
        if self.centrals_flag:
            self.taudict_keys = [self.all_key,
                                 self.clean_key,
                                 self.backsplash_z0_flag]
        else:
            self.taudict_keys = [self.backsplash_prev_flag,
                                 self.preprocessed_flag,
                                 self.clean_key]
                
        return


def argparse_Config(Config):
    """
    Parse all command line arguments and update them in the Config.
    """
    description = ('Pipeline for running all analysis scripts related to \n' +
                   'computing the RSP in TNG galaxies.')
    parser = argparse.ArgumentParser(description=description)
    
    # general flags
    parser.add_argument('--sim', default=None, type=str,
                        help='which simulation to use for the analysis.')
    
    parser.add_argument('--TNGCluster-flag', default=None, type=bool,
                        help='analysis for TNG-Cluster')
    parser.add_argument('--zooniverse-flag', default=None, type=bool,
                        help='analysis using CJF zooniverse results')
    parser.add_argument('--centrals-flag', default=None, type=bool,
                        help='analysis for central galaxies')
    parser.add_argument('--tracers-flag', default=None, type=bool,
                        help='use tracer post-processing catalogs in analysis.')
    parser.add_argument('--allsubhalos-flag', default=None, type=bool,
                        help='use all subhalos in the simulation.')
    # mp flags
    parser.add_argument('--mp-flag', default=None, type=bool,
                        help='use multiprocessing for analysis.')
    parser.add_argument('--Nmpcores', default=None, type=int,
                        help='Number of cores to use for multiprocessing tasks.')

    # hard coded values
    parser.add_argument('--max-snap', default=None, type=int,
                        help='max snap for merger trees and analysis.')
    parser.add_argument('--min-snap', default=None, type=int,
                        help='min snap for merger trees and analysis.')
    parser.add_argument('--first-snap', default=None, type=int,
                        help='first snap for running the tracers.')
    parser.add_argument('--last-snap', default=None, type=int,
                        help='last snap for running the tracers.')
    parser.add_argument('--tlim', default=None, type=float,
                        help='temperature limit between cold and hot gas.')
    parser.add_argument('--jellyscore-min', default=None, type=int,
                        help='minimum score to be considered a jellyfish galaxy')

    # which types of analysis should be run
    parser.add_argument('--SubfindIndices', default=None, type=bool,
                        help='flag to run Create_SubfindIndices.py.')
    parser.add_argument('--SubfindGasRadProf', default=None, type=bool,
                        help='flag to run Create_SubfindGasRadProf.py.')
    parser.add_argument('--run-SGRP', default=None, type=bool,
                        help='flag to run the main analysis in SubfindGasRadProf.py.')
    parser.add_argument('--run-SGRP-PP', default=None, type=bool,
                        help='flag to run the post processing of SGRP.')
    parser.add_argument('--SubfindSnapshot', default=None, type=bool,
                        help='flag to run Create_SubfindSnapshot_Flags.py.')
    parser.add_argument('--run-SS', default=None, type=bool,
                        help='flag to run main analysis in Create_SS_Flags.py.')
    parser.add_argument('--run-SS-PP', default=None, type=bool,
                        help='flag to run post-processing of Create_SS_Flags.py.')
    parser.add_argument('--TracerTracks', default=None, type=bool,
                        help='flag to run Create_TracerTracks.py.')
    parser.add_argument('--track-tracers', default=None, type=bool,
                        help='flag to run track_tracers().')
    parser.add_argument('--find-tracers', default=None, type=bool,
                        help='flag to run find_unmatched_tracers().')
    parser.add_argument('--CleanSubfindGasRadProf', default=None, type=bool,
                        help='flag to run Clean_SubfindGasRadProf.py.')
    parser.add_argument('--run-cleanSGRP', default=None, type=bool,
                        help='flag to run clean_subfindGRP().')
    parser.add_argument('--run-createtau', default=None, type=bool,
                        help='flag to run create_taudict().')

    args = vars(parser.parse_args())
    for key in args.keys():
        if args[key]:
            Config[key] = args[key]
        
    return Config
                        

def return_outdirec_outfname(Config):
    """
    given the simulation and flags, determine the directory and GRP filename
    """
    
    outdirec = '../Output/%s_subfindGRP/'%Config.sim
    if (Config.zooniverse_flag):
        outfname = 'zooniverse_%s_%s_branches.hdf5'%(Config.sim, Config.zooniverse_key)
        #outdirec = '../Output/zooniverse/'
    elif Config.centrals_flag:
        outfname = 'central_subfind_%s_branches.hdf5'%(Config.sim)
    elif Config.allsubhalos_flag:
        outfname = 'all_subfind_%s_branches.hdf5'%(Config.sim)
    else:
        outfname = 'subfind_%s_branches.hdf5'%(Config.sim)
        
    if (os.path.isdir(outdirec)):
        print('Directory %s exists.'%outdirec)
        if os.path.isfile(outdirec + outfname):
            print('File %s exists. Overwriting.'%(outdirec+outfname))
            return outdirec, outfname
        else:
            print('File %s does not exists. Writing.'%(outdirec+outfname))
            return outdirec, outfname
    else:
        print('Directory %s does not exist. Creating it now.'%outdirec)
        os.system('mkdir %s'%outdirec)
        return outdirec, outfname
        
        
def return_taufname(Config):
    """ given the simulation and flags, determine the tau filename """
    if Config.zooniverse_flag:
        return 'zooniverse_%s_%s_clean_tau.hdf5'%(Config.sim, Config.zooniverse_key)
    elif Config.centrals_flag:
        return 'central_subfind_%s_tau.hdf5'%(Config.sim)
    elif Config.allsubhalos_flag:
        return 'all_subfind_%s_clean_tau.hdf5'%(Config.sim)
    else:
        return 'subfind_%s_tau.hdf5'%(Config.sim)

        
def return_Mstar_lolim(Config):
    """ given the simulation, determine the minimum resolved Mstar mass"""
    sim = Config.sim
    if 'TNG50' in sim:
        res = 10.**(8.3)
    elif 'TNG100' in sim:
        res = 10.**(9.5)
    elif 'TNG300' in sim:
        res = 10.**(10)
    elif 'L680n8192TNG' in sim:
        return 10.**(10)
    else:
        raise ValueError('sim %s not recognized.'%sim)
        
    if sim == 'TNG50-4':
        return 10.**(10)
    
    for i in range(1,5):
        if '-%d'%i in sim:
            Mstar_lolim = res * 8**(i-1)
            break
        elif i == 4:
            raise ValueError('sim %s not recongized.'%sim)
            
    return Mstar_lolim
 

def initialize_allsubhalos(Config):
    """
    Create a list of the subfindIDs for all subhalos in the simulation.
    """
    
    Nsubhalos = il.groupcat.loadSubhalos(Config.basePath, Config.max_snap, fields='SubhaloGrNr').size
    SubfindIDs = np.arange(Nsubhalos)
    SnapNums = np.ones(SubfindIDs.size, dtype=int) * 99
    
    return SnapNums, SubfindIDs
    

def initialize_subfindindices(Config):
    """
    Define SubfindIDs and SnapNums to be tracked.
    Returns all z=0 satellites with Mstar > Mstar_lolim
    and SubhaloFlag == 1.
    Returns SnapNums, SubfindIDs
    """
    
    basePath = Config.basePath
    star_ptn = Config.star_ptn
    Mstar_lolim = Config.Mstar_lolim
    h = Config.h
    
    subhalo_fields = ['SubhaloFlag', 'SubhaloMassInRadType']
    subhalos = il.groupcat.loadSubhalos(basePath, 99, fields=subhalo_fields)

    halo_fields = ['GroupFirstSub']
    GroupFirstSub = il.groupcat.loadHalos(basePath, 99, fields=halo_fields)

    Mstar = subhalos['SubhaloMassInRadType'][:,star_ptn] * 1.0e10 / h
    subfind_indices = np.where((subhalos['SubhaloFlag']) & (Mstar >= Mstar_lolim))[0]
    indices = np.isin(subfind_indices, GroupFirstSub)
    SubfindIDs = subfind_indices[~indices]
    SnapNums = np.ones(SubfindIDs.size, dtype=int) * 99

    return SnapNums, SubfindIDs


def initialize_central_subfindindices(Config):
    """
    Define SubfindIDs and SnapNums to be tracked.
    Returns the most massive z=0 central subhalos.
    """
    
    halo_fields = ['Group_M_Crit200','GroupFirstSub']
    halos = il.groupcat.loadHalos(Config.basePath, 99, fields=halo_fields)
    M200c = halos['Group_M_Crit200'] * 1.0e10 / Config.h
    indices = M200c >= 10.0**(11.5)

    GroupFirstSub = halos['GroupFirstSub']
    SubfindIDs = GroupFirstSub[indices]
    SnapNums = np.ones(SubfindIDs.size, dtype=int) * 99

    return SnapNums, SubfindIDs
    
    
def initialize_TNGCluster_subfindindices(Config):
    """
    Define the SubfindIDs at z=0 to be tracked.
    """
    
    basePath = Config.basePath
    max_snap = Config.max_snap
    star_ptn = Config.star_ptn
    h = Config.h
    Mstar_lolim = Config.Mstar_lolim
    centrals_flag = Config.centrals_flag
    
    # load all halos and find the primary zoom target IDs
    halo_fields = ['Group_M_Crit200', 'GroupFirstSub', 'GroupPrimaryZoomTarget']
    halos = il.groupcat.loadHalos(basePath, max_snap, fields=halo_fields)
    haloIDs = np.where(halos['GroupPrimaryZoomTarget'])[0]
    GroupFirstSub = halos['GroupFirstSub'][haloIDs]
    
    print('There are %d primary zoom targets in %s.'%(haloIDs.size, Config.sim))
    
    # load all subhalos and find which ones:
    # 1) are z=0 satellites of primary zooms
    # 2) have Mstar(z=0) > Mstar_lolim
    subhalo_fields = ['SubhaloGrNr', 'SubhaloMassInRadType']
    subhalos = il.groupcat.loadSubhalos(basePath, max_snap, fields=subhalo_fields)
    subhalo_indices_massive = subhalos['SubhaloMassInRadType'][:,star_ptn] * 1.0e10 / h > Mstar_lolim
    
    _, subhalo_match_indices = ru.match3(haloIDs, subhalos['SubhaloGrNr'][subhalo_indices_massive])
    
    # remove the central galaxies
    subhaloIDs = np.where(subhalo_indices_massive)[0][subhalo_match_indices]
    isin = np.isin(subhaloIDs, GroupFirstSub, assume_unique=True)
    
    if centrals_flag:
        subfindIDs = subhaloIDs[isin]
    else:
        subfindIDs = subhaloIDs[~isin]
        
    snaps = np.ones(subfindIDs.size, dtype=subfindIDs.dtype) * max_snap
    
    return snaps, subfindIDs

    
def initialize_zooniverseindices(Config):
    """
    Load all zooniverse output catalogs for the given simulation and determine
    which galaxies have been inspected at multiple snapshots. Then tabulates the last
    snapshot at which the galaxy was inspected, and the subfindID at that snapshot.
    Returns SnapNums, SubfindIDs
    """
    
    indirec = '../IllustrisTNG/%s/postprocessing/Zooniverse_CosmologicalJellyfish/'%Config.sim
    infname = 'jellyfish.hdf5'
    
    with h5py.File(indirec + infname, 'r') as inf:
        snapnums = inf['Branches_SnapNum_LastInspect'][:]
        subfindIDs = inf['Branches_SubfindID_LastInspect'][:]
        
        inf.close()
    
    return snapnums, subfindIDs

 
    
fname = 'config.yaml'
config_dict = Configuration.from_yaml(fname)
Config = Configuration(config_dict)
Config.add_vals()


In [ ]:
Config

In [ ]:
### import modules
import illustris_python as il
import numpy as np
import h5py
import rohr_utils as ru 
import os
import multiprocessing as mp

In [ ]:
def init_result(Config):
    """
    Initialize the result for a single subhalo.
    """
    
    keys = Config.subfindsnapshot_flags
    
    false_return = np.zeros(Config.SnapNums.size, dtype=int)
    inval_return = false_return.copy() - 1
    float_return = false_return.copy() - 1.

    init_result = {}
    for key in keys:
        if key == Config.in_tree_key:
            init_result[key] = false_return.copy()
        elif key == Config.host_m200c_key:
            init_result[key] = float_return.copy()
        else:
            init_result[key] = inval_return.copy()
            
    return init_result


def return_outdirec_outfname(Config, snapshotflags=True):
    """
    Helper function to determine the directory and filename for the outputs.
    """
    
    outdirec = '../Output/%s_subfindsnapshotflags/'%(Config.sim)
    if snapshotflags:
        outfname = 'subfindsnapshot_flags.hdf5'
    else:
        outfname = 'subfind_flags.hdf5'

    if (os.path.isdir(outdirec)):
        print('Directory %s exists.'%outdirec)
        if os.path.isfile(outdirec + outfname):
            print('File %s exists. Overwriting.'%(outdirec+outfname))
        else:
            print('File %s does not exists. Writing.'%(outdirec+outfname))
    else:
        print('Directory %s does not exist. Creating it now.'%outdirec)
        os.system('mkdir %s'%outdirec)
    
    return outdirec, outfname


In [ ]:
def initialize_result(Config):
    """
    Create a list of all subfindIDs of interest, namely those
    defined in Create_SubfindIndices. However, we want to use the subfindID
    as the index into the final result. So we create the result to be of size
    all subfindIDs (with all -1 values), and later calculate the flags only
    for the subhalos of interest.
    """
    
    subfindIDs = Config.SubfindIDs
    if Config.zooniverse_flag:
        grp_dict = h5py.File(Config.outdirec + Config.GRPfname, 'r')
        SubfindIDs = []
        for key in grp_dict.keys():
            group = grp_dict[key]
            indices = group['SubfindID'][:] != -1
            if (group['SnapNum'][indices].max() == 99):
                SubfindIDs.append(group['SubfindID'][0])
        subfindIDs = np.array(SubfindIDs)
        subfindIDs.sort()

    SnapNums = Config.SnapNums
    subfind_flags = Config.subfindsnapshot_flags
    
    # use the full catalog for the number of subhalos
    Nsubhalos = il.groupcat.loadSubhalos(Config.basePath, Config.max_snap, fields='SubhaloGrNr').size
    
    # initlaize and fill finalize result
    result = {}
    # check if the files already exist, and if so, overwrite
    subfindsnapshot_outdirec, subfindsnapshot_outfname = return_outdirec_outfname(Config, snapshotflags=True)
    if os.path.isfile(subfindsnapshot_outdirec + subfindsnapshot_outfname):
        with h5py.File(subfindsnapshot_outdirec + subfindsnapshot_outfname, 'r') as f:
            group = f['group']
            for key in group.keys():
                result[key] = group[key][:]
            f.close()
        return subfindIDs, result

    # file does not exist, so initialize result
    for flag in subfind_flags:
        if flag == Config.host_m200c_key:
            result[flag] = np.zeros((Nsubhalos, SnapNums.size), dtype=float) - 1.
        else:
            result[flag] = np.zeros((Nsubhalos, SnapNums.size), dtype=int) - 1
    
    return subfindIDs, result


In [ ]:
    # check the output directory and fname
    subfindsnapshot_outdirec, subfindsnapshot_outfname = return_outdirec_outfname(Config, snapshotflags=True)
            
    # initialize the subfindIDs of interest and final result shape
    subfindIDs, result = initialize_result(Config)


In [ ]:
Config.run_SS, Config.run_SS_PP

In [ ]:
        print('run_SS_PP: Number of subhalos of interest: %d'%subfindIDs.size)


In [ ]:
    Nsnaps_PP = Config.Nsnaps_PP
    M200c0_lolim_PP = Config.M200c0_lolim_PP

    # load the catalogs
    subfindsnapshot_outdirec, subfindsnapshot_outfname = return_outdirec_outfname(Config, snapshotflags=True)
    f = h5py.File(subfindsnapshot_outdirec + subfindsnapshot_outfname, 'r')
    group = f['group']

    central_key = Config.central_key
    in_tree_key = Config.in_tree_key
    host_m200c_key = Config.host_m200c_key
    in_z0_host_key = Config.in_z0_host_key
    
    classified_flag = Config.classified_flag
    central_z0_flag = Config.central_z0_flag
    backsplash_z0_flag = Config.backsplash_z0_flag
    backsplash_prev_flag = Config.backsplash_prev_flag
    preprocessed_flag = Config.preprocessed_flag
    flags = Config.subfind_flags
             
    # initialize result
    result = {}
    # first check if the files already exist, and if so, append to them
    subfindsnapshot_outdirec, subfindsnapshot_outfname = return_outdirec_outfname(Config, snapshotflags=False)
    if os.path.isfile(subfindsnapshot_outdirec + subfindsnapshot_outfname):
        with h5py.File(subfindsnapshot_outdirec + subfindsnapshot_outfname, 'r') as inf:
            ingroup = inf['group']
            for key in ingroup.keys():
                result[key] = ingroup[key][:]
            inf.close()
    else:
        for flag in flags:
            result[flag] = np.zeros(len(group[in_tree_key]), dtype=int)
            if flag != classified_flag:
                result[flag] -= 1


In [ ]:
i = 1
subfindID = subfindIDs[i]

In [ ]:
        # we only care about snapshots when the subhalo was identified in the merger trees
        in_tree = group[in_tree_key][subfindID]
        intree_indices = in_tree == 1

In [ ]:
intree_indices[intree_indices].size < Nsnaps_PP

In [ ]:
        result[classified_flag][subfindID] = 1
        
        central = group[central_key][subfindID][intree_indices] == 1
        in_z0_host = group[in_z0_host_key][subfindID][intree_indices] == 1
        host_m200c = group[host_m200c_key][subfindID][intree_indices] >= M200c0_lolim_PP


In [ ]:
        if (central[0] == 1):
            result[central_z0_flag][subfindID] = 1
            # yes! Is the subhalo a backsplash galaxy?
            backsplash_indices = ~central & ~in_z0_host & host_m200c
            backsplash_check = [True] * Config.Nsnaps_PP
            if (ru.is_slice_in_list(backsplash_check, backsplash_indices)):
                result[backsplash_z0_flag][subfindID] = 1
            else:
                result[backsplash_z0_flag][subfindID] = 0
  

In [ ]:
result[backsplash_z0_flag][subfindID]

In [ ]:
        # no, the subhalo is a satellite at z=0
        result[central_z0_flag][subfindID] = 0
        # was the subhalo pre-processed?
        preprocessed_indices = ~central & ~in_z0_host & host_m200c
        preprocessed_check = [True] * Nsnaps_PP
        if (ru.is_slice_in_list(preprocessed_check, preprocessed_indices)):
            result[preprocessed_flag][subfindID] = 1
        else:
            result[preprocessed_flag][subfindID] = 0
  

In [ ]:
result[preprocessed_flag][subfindID]

In [ ]:
        # was the subhalo previously a backsplash?
        # first, find the first time that the subhalo spent at least Nsnaps_PP in a massive host
        satellite_indices = ~central & host_m200c
        satellite_check = [True] * Nsnaps_PP
        satellite_indices_bool = ru.where_is_slice_in_list(satellite_check, satellite_indices)
        if any(satellite_indices_bool):
            # from this first time that the subhalo was a satellite, find the index of
            # the last conescutive snapshot.
            end_index = np.where(satellite_indices_bool)[0].max()
            
            if end_index > 0:
                # after this time, was the galaxy a central for at least Nsnaps_PP?
                central_indices = central[:end_index]
                central_check = [True]
                if central_indices.size >= Nsnaps_PP:
                    if ru.is_slice_in_list(central_check, central_indices):
                        result[backsplash_prev_flag][subfindID] = 1
        else:
            result[backsplash_prev_flag][subfindID] = 0
 

In [ ]:
any(satellite_indices_bool)

In [ ]:
result[backsplash_prev_flag][subfindID]

In [ ]:
ru.where_is_slice_in_list(satellite_check, satellite_indices)

In [ ]:
np.array([~central, host_m200c]).T